## Vector-valued function with Random Fourier Features

### Imports

In [449]:
import numpy as np
from numpy.random import randn

### Values

In [450]:

dimension = 2
sigma = 10
sigma2 = 2*sigma**2

# Random Fourier features
m = 20 # number of samples
w = np.array([randn(m) / sigma, randn(m) / sigma]) # w is the same size as the dimensions times the number of samples
b = np.random.rand(m)*2*np.pi # b is the same size as number of samples

n = 100 # number of points

x1 = randn(n)/2
x2 =  randn(n)/2
x_vector = np.array([x1, x2])
# x_vector = np.block([x1,x2])

y1 = np.cos(x1) + randn(n) * 1e-1
y2 = np.sin(x2) + randn(n) * 1e-1


#y1 = 1+np.sin(x1/10) + x1**2
#y2 = 0.5 * x2**2 + np.exp(x2)

y = np.block([y1, y2])
print(y)

#print(y1, y2)
lam = 0.000001 # Regularisaton parameter

[ 7.03000817e-01  7.92898004e-01  9.53317048e-01  3.09500831e-01
  7.78155631e-01  1.26259459e+00  6.21643668e-01  9.85271226e-01
  3.60270208e-01  1.07313158e+00  8.95652162e-01  9.68365691e-01
  9.92587240e-01  3.96795542e-01  1.03334465e+00  9.64569423e-01
  5.00731448e-01  7.92524756e-01  9.16192502e-01  1.09806795e+00
  8.17813400e-01  7.72053033e-01  1.04203936e+00  8.13668597e-01
  5.95413982e-01  1.04863639e+00  9.13596553e-01  9.96535454e-01
  1.01393297e+00  5.39502753e-01  1.06078798e+00  8.90872156e-01
  7.36464827e-01  1.14733806e+00  9.07440539e-01  5.91201179e-02
  7.11405672e-01  1.02101913e+00  1.00610424e+00  1.08450400e+00
  7.53584926e-01  8.86239376e-01  9.73309072e-01  8.73504257e-01
  9.35848513e-01  1.00483567e+00  1.18204028e+00  1.01407112e+00
  8.98750225e-01  8.05579011e-01  1.00939059e+00  1.05806518e+00
  5.44931406e-01  7.94236569e-01  8.24711907e-01  1.11247649e+00
  1.09365536e+00  9.93948213e-01  7.44624452e-01  9.87874092e-01
  4.86549661e-01  8.72804

### Gaussian Kernel

In [451]:
def k_gauss(x, z):
    k = np.exp(-((np.linalg.norm(x-z))**2)/(sigma2))
    return k

def k_matrix(x, z, dim):
    k = k_gauss(x, z)
    k_matrix = k *np.eye(dim)
    return k_matrix

def K_gauss(x, n, dim):
    K = np.zeros((dim*n, dim*n))
    for i in range(0, n):
        for j in range(0, n):
            k = k_gauss(x[:,i], x[:,j])
            K[i*dim][j*dim] = k
            K[i*dim+1][j*dim+1] = k
    return K

K = K_gauss(x_vector, n, dimension)
print(K)

[[1.         0.         0.9939162  ... 0.         0.98021792 0.        ]
 [0.         1.         0.         ... 0.99486918 0.         0.98021792]
 [0.9939162  0.         1.         ... 0.         0.97891364 0.        ]
 ...
 [0.         0.99486918 0.         ... 1.         0.         0.99199536]
 [0.98021792 0.         0.97891364 ... 0.         1.         0.        ]
 [0.         0.98021792 0.         ... 0.99199536 0.         1.        ]]


#### Alpha

In [452]:
# Returns a vector with length dim * n
def alpha(x, y, lam, n, dim):
    K = K_gauss(x, n, dim)
    alpha_vector = np.linalg.inv((K + lam*np.eye(dim*n))) @ y
    alpha_reshape = [alpha_vector[::2], alpha_vector[1::2]]
    return np.array(alpha_reshape)

alpha_ = alpha(x_vector, y, lam, n, dimension)
print(alpha_)

[[ 2.94486682e+05  4.28674899e+05  2.80698199e+05  6.10860160e+04
  -1.21768215e+05  4.05530359e+05  6.03054838e+05  5.89536727e+05
   4.84872031e+05  4.84106237e+05  2.43676859e+05  4.27036972e+05
   3.65142493e+05  4.98202789e+05  8.50815242e+05  6.02751487e+05
   2.70325161e+05  6.60098192e+05  2.09459093e+05  5.39062633e+05
   4.89258980e+05  4.87179572e+05  5.89113116e+05  5.71558185e+05
   4.04333919e+05  6.06472605e+05  1.89298537e+05  3.77796894e+05
   6.77006654e+05  3.49755666e+05 -7.02239789e+04  5.98646248e+05
   7.12589567e+05  4.14638839e+05  4.92394055e+05  4.37333235e+05
   5.52375180e+05  4.05131598e+05  6.82148174e+05  3.86418452e+05
   6.82379020e+05  7.64753761e+05  2.84719342e+05  2.65360907e+05
   4.49611165e+05  3.72095051e+05  2.93800822e+05  5.58773659e+05
   3.88313096e+05  5.88746375e+05 -7.14129414e+05 -3.92356563e+05
  -4.63181336e+05 -6.50777579e+05 -9.06153118e+05 -4.03611454e+04
  -1.02601958e+06 -1.17260864e+06 -8.50472423e+05 -4.70495886e+05
  -9.97752

#### Estimated function

In [453]:
def estiamted_function(alpha, x, n, dim):
    function = np.zeros((2, n))
    for i in range(n):
        sum1 = np.zeros((2))
        x_i = x[:, i]
        for j in range(n):
            k = k_gauss(x_i, x[:, j])
            kernel = k*np.eye(dim)
            sum1 = np.add(sum1, kernel@alpha[:, j])
        function[:, i] = sum1
    return function

estiamted_function(alpha_, x_vector, n, dimension)


array([[ 0.40851413,  0.52464216,  0.49745744,  0.56055765,  0.48203842,
         0.4901218 ,  0.3895324 ,  0.44380793,  0.01585942,  0.43208627,
         0.57413654,  0.61500238,  0.23027149,  0.41539377,  0.16311773,
         0.4580365 ,  0.46613966,  0.24734235,  0.50194658,  0.46704161,
         0.26432595,  0.48612951,  0.3467354 ,  0.6104821 ,  0.49441631,
         0.40291799,  0.35563288,  0.44691501,  0.41664871,  0.39486879,
         0.55677364,  0.35895237,  0.43705047,  0.54117962,  0.52668755,
         0.46607252,  0.49165286,  0.6006804 ,  0.37359249,  0.47128569,
         0.37792936,  0.32655317,  0.54061328,  0.48125035,  0.57416833,
         0.47923947,  0.54355668,  0.35654605,  0.49389142,  0.33131756,
         0.62021782,  0.33394519,  0.38942416,  0.32299398,  0.38140408,
         0.27204349,  0.33415626,  0.34354224,  0.35615759,  0.40337697,
         0.55967457,  0.37717326,  0.26898509,  0.45200192,  0.12047851,
         0.54728262,  0.46754359,  0.38505915,  0.4

### With Random Fourier Features

In [454]:
def psi(x, w, b):
    print(np.sqrt(2) * np.cos(w.T * x + b))
    return np.sqrt(2) * np.cos(w.T * x + b)

def psi_vector(x, w, b, m):
    psi_vector = np.zeros(m)
    for i in range(m):
        print(psi(x[:, i], w[:, i], b[i]))
        psi_vector[i] = psi(x[:, i], w[:, i], b[i])
    return np.array(psi_vector)

def capital_psi(psi_vector, dim):
    return np.kron(psi_vector, dim)

psi_ = psi_vector(x_vector, w, b, m)
capital_psi(psi_, dimension)

[-1.37170278 -1.38631549]
[-1.37170278 -1.38631549]
[-1.37170278 -1.38631549]


ValueError: setting an array element with a sequence.